Please install the necessary libraries using your terminal/shell in your env before running the code. You can check all the libraries mentioned in the next code block. To install any library you can use either one of the code mentioned below

pip install library_name

pip3 install library_name

In [2]:
import os
import glob
import yaml
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import json
from matplotlib.backends.backend_pdf import PdfPages
import warnings
import numpy as np
from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image

warnings.filterwarnings('ignore')

class_name = { 0: 'Mass',
1: 'Spiculation',
2: 'Suspicious Calcification',
3: 'Architectural Distortion',
4: 'Asymmetry',
5: 'Focal Asymmetry',
6: 'Skin Thickening',
7: 'Global Asymmetry',
8: 'Suspicious Lymph Node',
9: 'Skin Retraction',
10: 'Nipple Retraction'
}

color_dict = {
    0: (255, 0, 0),      # Red
    1: (0, 255, 0),      # Green
    2: (0, 0, 255),      # Blue
    3: (255, 255, 0),    # Yellow
    4: (255, 165, 0),    # Orange
    5: (128, 0, 128),    # Purple
    6: (0, 255, 255),    # Cyan
    7: (255, 192, 203),  # Pink
    8: (128, 128, 0),    # Olive
    9: (0, 0, 0),        # Black
    10: (169, 169, 169)  # Dark Grey
}

output_labels_dir = "results/labels"  # Directory to save ground truth images
output_pred_dir = "results/predictions"  # Directory to save predicted images

Mention dataset.yaml file path and run

In [3]:
path = "dataset.yaml"

def remove_cache_files(directory):
    cache_files = glob.glob(os.path.join(directory, "*.cache"))
    for cache_file in cache_files:
        os.remove(cache_file)
        print(f"Removed: {cache_file}")
        
with open(path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)

remove_cache_files(os.path.dirname(data_loaded['train']))
remove_cache_files(os.path.dirname(data_loaded['val']))

Removed: /home/rshah133/bcd/dataset/split_1/train/labels.cache
Removed: /home/rshah133/bcd/dataset/split_1/val/labels.cache


Change the model and hyperparameters.

* Epochs = keep minimum 200
* imgsz = change acc to your model assigned
* patience = number of epochs model will check for accuracy improvement and then stop if accuracy is not improving
* set device = [0,1] if you have 2 GPUs requested and device = 0 if only 1 GPU/CPU
* save = True, this will help you save the model checkpoints on its own 
* save_period = 10, this is the number of epochs after which model will save the checkpoints
* resume = True, if kernel crashes, model will continue training from the previous checkpoint
* iou = keep iou between 0.5-0.7 and check for best metrics (hyperparameter to play with)
* optimzer  = AdamW is the best for our model from what I have noticed. you can experiment if you want
* learning rate, momentum, and weight decay are few more hyperparamerts you can play with

In [4]:
model = YOLO('yolov8s.yaml')
results = model.train(data = path, epochs = 200, imgsz = 1024, batch = 10, name = 'checkpoint', device = [0,1], patience = 25, save = True, save_period = 10, exist_ok = True, resume = True, iou = 0.5, optimizer = 'AdamW')

New https://pypi.org/project/ultralytics/8.3.69 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.68 🚀 Python-3.11.6 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-80GB, 81156MiB)
                                                      CUDA:1 (NVIDIA A100-SXM4-80GB, 81156MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=dataset.yaml, epochs=200, time=None, patience=25, batch=10, imgsz=1024, save=True, save_period=10, cache=False, device=[0, 1], workers=8, project=None, name=checkpoint, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=None, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, aug

E0000 00:00:1738121681.150214  923098 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738121681.153136  923098 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=11

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

E0000 00:00:1738121772.290063  923677 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738121772.294071  923677 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorBoard: Start with 'tensorboard --logdir runs/detect/checkpoint', view at http://localhost:6006/
Overriding model.yaml nc=80 with nc=11
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


...

    131/200       3.3G      2.498      2.845      2.311         67       1024: 100%|██████████| 20/20 [00:02<00:00,  8.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00, 14.24it/s]


                   all         25        125      0.697     0.0263     0.0343     0.0144


    132/200      3.53G        2.8      3.157       2.58         50       1024:   5%|▌         | 1/20 [00:00<00:02,  7.95it/s]


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    132/200      3.55G      2.553      2.896      2.379         38       1024:  85%|████████▌ | 17/20 [00:02<00:00,  8.45it/s]

Implemented Tensorboard. Code will redirect you to a link which should be opened in **Google Chrome** if nothing is getting displayed on safari.  You can choose not to run this code block if you want to not see the dashboard and save time.

In [4]:
#!kill 1869322
%load_ext tensorboard
%tensorboard --logdir runs/detect/checkpoint

Reusing TensorBoard on port 6008 (pid 1877066), started 0:29:56 ago. (Use '!kill 1877066' to kill it.)

Mention test_dataset.yaml file path and run

In [4]:
test_path = "test_dataset.yaml"

with open(test_path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)

remove_cache_files(os.path.dirname(data_loaded['train']))
remove_cache_files(os.path.dirname(data_loaded['val']))

Removed: /home/rshah133/bcd/dataset/test/labels.cache


Predicting on test data. Please set the prediction_progress.json to 0 by opening everytime when you want to predict on the test data from the start or it will resume from where it stopped last

In [ ]:
progress_file = 'prediction_progress.json'

def save_progress(current_index):
    with open(progress_file, 'w') as f:
        json.dump({'last_processed': current_index}, f)

def load_progress():
    if os.path.exists(progress_file):
        with open(progress_file, 'r') as f:
            return json.load(f)['last_processed']
    return 0

# Loading the best model
best_model = YOLO('runs/detect/checkpoint/weights/best.pt')

test_img_path = data_loaded['val']

start_index = load_progress()

chunk_size = 48  # Adjust based on your available memory
for i in range(start_index, len(test_img_path), chunk_size):
    chunk_end = min(i + chunk_size, len(test_img_path))
    current_chunk = test_img_path[i:chunk_end]
    
    try:
        results = best_model.predict(source = current_chunk, save = True, save_txt = True, conf = 0.10, batch = chunk_size//4, stream = True)
        
        # code for post - processing results for later
        for r in results:
            pass  
            
        save_progress(chunk_end)
        
    except Exception as e:
        save_progress(i)
        print(f"Prediction stopped at image {i}. Progress saved.")
        raise e


/home/rshah133/bcd/dataset/test/images

image 1/763 /home/rshah133/bcd/dataset/test/images/001ade2a3cb53fd808bd2856a0df5413.jpg: 1024x1024 (no detections), 2.8ms
image 2/763 /home/rshah133/bcd/dataset/test/images/002074412a8fc178c271fb93b55c3e29.jpg: 1024x1024 2 Masss, 5 Spiculations, 2.8ms
image 3/763 /home/rshah133/bcd/dataset/test/images/005918369ec07b1aed37d1dd78bc57fe.jpg: 1024x1024 1 Mass, 2 Spiculations, 2.8ms
image 4/763 /home/rshah133/bcd/dataset/test/images/00857417d07096982013956033da1f75.jpg: 1024x1024 (no detections), 2.8ms
image 5/763 /home/rshah133/bcd/dataset/test/images/008bc6050f6d31fc255e5d87bcc87ba2.jpg: 1024x1024 1 Mass, 2 Spiculations, 2.8ms
image 6/763 /home/rshah133/bcd/dataset/test/images/00be38a5c0566291168fe381ba0028e6.jpg: 1024x1024 1 Spiculation, 2.8ms
image 7/763 /home/rshah133/bcd/dataset/test/images/00ec2be128f964da6f0b0ba179c4d138.jpg: 1024x1024 1 Mass, 1 Spiculation, 2.8ms
image 8/763 /home/rshah133/bcd/dataset/test/images/01599597388f3185563decc34945f

In [6]:
# best_model = YOLO('runs/detect/checkpoint/weights/best.pt')
# test_img_path = data_loaded['val']
# results = best_model.predict(source = test_img_path, save = True,  save_txt = True,  conf = 0.10, batch = 16)

Run this cell as it is and take screenshot of the output. This code is for calculating the metrics for test data

u can play with conf keep between 0.05-0.2 (for our use case). lower value means it predicts more but with less accuracy. 

In [13]:
metrics = best_model.val(data = test_path, conf = 0.10)

Ultralytics 8.3.68 🚀 Python-3.11.6 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-80GB, 81158MiB)


val: Scanning /home/rshah133/bcd/dataset/test/labels.cache... 763 images, 34 backgrounds, 0 corrupt: 100%|██████████| 763/763 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 48/48 [00:08<00:00,  5.64it/s]


                   all        763       1860      0.124     0.0364     0.0252    0.00971
                  Mass        388        451      0.216      0.333      0.217     0.0831
           Spiculation        263        810     0.0923     0.0593     0.0406     0.0157
Suspicious Calcification        280        358     0.0516    0.00806     0.0201    0.00799
Architectural Distortion         59         59          0          0          0          0
             Asymmetry          8          8          0          0          0          0
       Focal Asymmetry         93         93          0          0          0          0
       Skin Thickening         31         33          0          0          0          0
      Global Asymmetry          2          2          0          0          0          0
 Suspicious Lymph Node         27         30          1          0          0          0
       Skin Retraction          3          4          0          0          0          0
     Nipple Retra

In [5]:
def get_prediction_dirs():
    base_dir = "runs/detect"
    
    predict_dirs = [d for d in os.listdir(base_dir) if d.startswith('predict')]
    predict_dirs.sort(key=lambda x: int(x.replace('predict', '')) if x != 'predict' else 0)
    
    return f"{base_dir}/{predict_dirs[-1]}/labels"

predictions_dir = get_prediction_dirs()
predictions_dir

'runs/detect/predict/labels'

In [ ]:
test_images_dir = data_loaded['val']
test_labels_dir = os.path.join(os.path.dirname(data_loaded['val']), 'labels')

# Create output directory if it doesn't exist
os.makedirs(output_labels_dir, exist_ok=True)
os.makedirs(output_pred_dir, exist_ok=True)

# Function to read YOLO format labels
def read_yolo_labels(label_file):
    labels = []
    with open(label_file, "r") as f:
        for line in f:
            parts = line.strip().split()
            class_id, x_center, y_center, width, height = map(float, parts[:5])
            confidence = float(parts[5]) if len(parts) > 5 else None
            labels.append((int(class_id), x_center, y_center, width, height, confidence))
    return labels

# Function to draw bounding boxes on images
def draw_boxes(image, boxes):
    h, w, _ = image.shape
    for box in boxes:
        class_id, x_center, y_center, width, height, confidence = box
        x_min = int((x_center - width / 2) * w)
        y_min = int((y_center - height / 2) * h)
        x_max = int((x_center + width / 2) * w)
        y_max = int((y_center + height / 2) * h)
        # Draw rectangle
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color_dict[class_id], 2)
    return image

# Loop through each image
for image_file in os.listdir(test_images_dir):
    if image_file.endswith((".jpg", ".png", ".jpeg")):
        base_name = os.path.splitext(image_file)[0]
        image_path = os.path.join(test_images_dir, image_file)

        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error loading {image_file}")
            continue

        # Read ground truth labels
        ground_truth_file = os.path.join(test_labels_dir, f"{base_name}.txt")
        ground_truth_boxes = read_yolo_labels(ground_truth_file) if os.path.exists(ground_truth_file) else []

        # Read prediction labels
        prediction_file = os.path.join(predictions_dir, f"{base_name}.txt")
        prediction_boxes = read_yolo_labels(prediction_file) if os.path.exists(prediction_file) else []

        # Draw ground truth (green) and predictions (blue)
        image_with_boxes_gt = draw_boxes(image.copy(), ground_truth_boxes)
        image_with_boxes_pt = draw_boxes(image.copy(), prediction_boxes)

        # Save annotated image
        gt_output_path = os.path.join(output_labels_dir, image_file)
        cv2.imwrite(gt_output_path, image_with_boxes_gt)

        pt_output_path = os.path.join(output_pred_dir, image_file)
        cv2.imwrite(pt_output_path, image_with_boxes_pt)

print("Done saving the predictions")

Done saving the predictions


Code to combine images in a single pdf. This code take upto 20mins to execute

In [187]:
%matplotlib inline

output_labels_file_names = sorted(os.listdir(output_labels_dir))
output_pred_file_names = sorted(os.listdir(output_pred_dir))
c = 0
pdf_file_path = "combined_images.pdf"

# model = YOLO('/home/rshah133/bcd/v8n_1024/runs/detect/checkpoint/weights/best.pt')
model = model.cpu()
target_layers = [model.model.model[-4]]
cam = EigenCAM(model, target_layers, task='od')


with PdfPages(pdf_file_path) as pdf:
    for label_file, pred_file in zip(output_labels_file_names, output_pred_file_names):
        label_img = cv2.imread(os.path.join(output_labels_dir, label_file))
        pred_img = cv2.imread(os.path.join(output_pred_dir, pred_file))
        
        img = cv2.resize(pred_img, (640, 640))
        rgb_img = img.copy()
        img = np.float32(img) / 255
        
        grayscale_cam = cam(rgb_img)[0, :, :]
        
        grayscale_cam_resized = cv2.resize(grayscale_cam, (img.shape[1], img.shape[0]))
        cam_image = show_cam_on_image(img, grayscale_cam_resized, use_rgb=True)
        
        fig, axs = plt.subplots(1, 3, figsize=(16, 6))
        axs[0].imshow(cv2.cvtColor(label_img, cv2.COLOR_BGR2RGB))
        axs[0].set_title('Ground Truth')
        axs[0].axis('off')
        axs[1].imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
        axs[1].set_title('Prediction')
        axs[1].axis('off')
        axs[2].imshow(cv2.cvtColor(cam_image, cv2.COLOR_BGR2RGB))
        axs[2].set_title('EigenCAM Explainable AI')
        axs[2].axis('off')
        
        c+=1
        pdf.savefig(fig)
        plt.close(fig)
    
print(f"{c} prediction images saved in {pdf_file_path}")


0: 1024x1024 (no detections), 954.1ms
Speed: 5.3ms preprocess, 954.1ms inference, 0.6ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 986.1ms
Speed: 5.3ms preprocess, 986.1ms inference, 0.5ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 967.6ms
Speed: 5.6ms preprocess, 967.6ms inference, 0.4ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 962.2ms
Speed: 6.4ms preprocess, 962.2ms inference, 0.5ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 (no detections), 973.9ms
Speed: 5.3ms preprocess, 973.9ms inference, 0.6ms postprocess per image at shape (1, 3, 1024, 1024)
5 prediction images saved in combined_images.pdf


In [7]:
# c = 0

# output_labels_file_names = sorted(os.listdir(output_labels_dir))
# output_pred_file_names = sorted(os.listdir(output_pred_dir))

# pdf_file_path = "combined_images.pdf"
# with PdfPages(pdf_file_path) as pdf:
#     for label_file, pred_file in zip(output_labels_file_names[:5], output_pred_file_names[:5]):
#         label_img = cv2.imread(os.path.join(output_labels_dir, label_file))
#         pred_img = cv2.imread(os.path.join(output_pred_dir, pred_file))

#         fig, axs = plt.subplots(1, 2, figsize=(16, 6))
#         fig.suptitle(f"Image: {label_file}", fontsize=12)
#         axs[0].imshow(cv2.cvtColor(label_img, cv2.COLOR_BGR2RGB))
#         axs[0].set_title('Ground Truth')
#         axs[0].axis('off')
#         axs[1].imshow(cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB))
#         axs[1].set_title('Prediction')
#         axs[1].axis('off')
#         plt.tight_layout()
#         c += 1
        
#         pdf.savefig(fig)
#         plt.close(fig)

# print(f"{c} prediction images saved in {pdf_file_path}")

5 prediction images saved in combined_images.pdf
